In [1]:
import pandas as pd
import talib as ta

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [8]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if future["volumeUsd24h"] > 1e7:
            markets.append(future["name"])

In [9]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "RetDevRatio", "MinReturns", "MaxReturns", 
                                "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio"])

for market in markets:
    df = ftx.get_historical_market_data(market, interval="4h", start_time="100 days ago")
    
    if len(df) < 600:
        continue
    
    profits = bt.backtest_dataframe(df)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

In [10]:
analysis_df = analysis_df.round(1)
analysis_df.to_csv("MarketAnalysis.csv", index=False)

In [11]:
calc_df = pd.read_csv("MarketAnalysis.csv")

In [12]:
calc_df

,Name,AvgReturns,StdDev,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio
0,1INCH-PERP,21.8,33.7,0.6,-4.0,155.2,-1.8,29.2,15.8
1,AAVE-PERP,9.6,21.9,0.4,-22.2,85.7,-7.5,20.1,2.7
2,ADA-PERP,9.0,12.4,0.7,-12.5,34.3,-4.3,14.2,3.3
3,ATOM-PERP,15.9,24.0,0.7,-9.6,113.7,-4.9,21.7,4.4
4,AXS-PERP,27.6,52.1,0.5,-6.6,231.0,-4.5,35.7,7.8
5,BCH-PERP,14.6,29.0,0.5,-5.7,132.3,-2.9,24.0,8.3
6,BNB-PERP,10.7,25.0,0.4,-5.2,135.7,-1.7,15.5,9.4
7,BTC-PERP,5.2,10.8,0.5,-10.2,46.2,-3.2,9.1,2.8
8,COMP-PERP,12.3,25.9,0.5,-10.8,125.7,-2.9,21.7,7.4
9,DOGE-PERP,24.0,40.5,0.6,-8.2,137.7,-4.5,40.5,9.1


In [14]:
bt.get_backtest_ranking(10, filename="MarketAnalysis.csv", sort_by_column="PosNegRetRatio")

'5/33'